# Proyecto 2 modelos

In [2]:
import pandas as pd
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import ndimage
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import cv2
from PIL import Image
from zipfile import ZipFile
from torchvision import transforms
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

In [3]:
data_VGG = pd.read_csv('train_filtrado.csv')

In [4]:
data_VGG.head()

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0


In [5]:
# Parámetros del modelo
vgg_output_size = 512  # Tamaño de la salida de la VGG16
gru_hidden_size = 128  # Tamaño del estado oculto de la GRU
gru_num_layers = 2  # Número de capas en la GRU
num_classes = 7  # Reemplaza con el número de clases en tu problema

In [6]:
class WeightedMultiLabelLogLoss(nn.Module):
    def __init__(self, weight=None):
        super(WeightedMultiLabelLogLoss, self).__init__()
        self.weight = weight

    def forward(self, input, target):
        """
        Computes the weighted multi-label logarithmic loss.

        Args:
            input (torch.Tensor): Predicted probabilities (output of the model).
                                Shape: (batch_size, num_classes)
            target (torch.Tensor): Target labels (ground truth).
                                Shape: (batch_size, num_classes)

        Returns:
            loss (torch.Tensor): Weighted multi-label logarithmic loss.
        """
        epsilon = 1e-15  # Small constant to avoid log(0)

        # Log loss
        log_loss = -target * torch.log(input + epsilon) - (1 - target) * torch.log(1 - input + epsilon)

        # Apply weights if provided
        if self.weight is not None:
            log_loss = log_loss * self.weight

        # Compute mean loss over samples and classes
        loss = log_loss.mean()

        return loss

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define la arquitectura del modelo combinado
class CombinedModel(nn.Module):
    def __init__(self, vgg_output_size, gru_hidden_size, gru_num_layers, num_classes):
        super(CombinedModel, self).__init__()
        self.vgg16 = models.vgg16(pretrained=True)
        self.vgg16.features[28] = nn.AvgPool2d(kernel_size=7, stride=1)  # Cambia el stride de la última capa de pooling
        self.vgg16.classifier = nn.Identity()  # Desactiva las capas clasificadoras

        self.gru = nn.GRU(vgg_output_size, gru_hidden_size, num_layers=gru_num_layers, batch_first=True)
        self.fc = nn.Linear(gru_hidden_size, num_classes)

    def forward(self, x):
        # Pasa cada imagen a través de VGG16 y obtén los feature maps
        vgg_outputs = []
        for image in x:
            vgg_output = self.vgg16(image)
            vgg_output = vgg_output.view(vgg_output.size(0), -1, vgg_output.size(1))
            vgg_outputs.append(vgg_output)

        # Concatena los feature maps de las imágenes a lo largo de la dimensión del tiempo (secuencia)
        gru_input = torch.cat(vgg_outputs, dim=0)  # Concatena en la dimensión 0 (batch_size x num_images, 512, height, width)

        # Reorganiza la entrada para que cada imagen sea una "secuencia" para la GRU
        gru_input = gru_input.view(-1, len(vgg_outputs), vgg_output_size)  # (batch_size, num_images, 512)

        # Pasa la secuencia de entrada a través de GRU
        gru_output, _ = self.gru(gru_input)

        # Pasa la última salida de la GRU a través de una capa completamente conectada
        output = self.fc(gru_output[:, -1, :])
        print("pase")
        return output


In [8]:
# # Define la arquitectura del modelo combinado con EfficientNet
# class CombinedModel(nn.Module):
#     def __init__(self, efficientnet_model, gru_hidden_size, gru_num_layers, num_classes):
#         super(CombinedModel, self).__init__()
#         self.efficientnet = efficientnet_model
#         self.efficientnet._avg_pooling = nn.AdaptiveAvgPool2d(1)  # Cambia el pooling a adaptativo para cualquier tamaño de entrada
#         self.efficientnet._fc = nn.Identity()  # Desactiva las capas completamente conectadas

#         # Obtén el tamaño de salida de EfficientNet (depende de la versión)
#         # Asume que la última convolución es 2D
#         dummy_input = torch.randn(1, 3, 224, 224)  # Suponiendo imágenes de 224x224
#         dummy_output = self.efficientnet.extract_features(dummy_input)
#         self.efficientnet_output_size = dummy_output.shape[1]

#         self.gru = nn.GRU(self.efficientnet_output_size, gru_hidden_size, num_layers=gru_num_layers, batch_first=True)
#         self.fc = nn.Linear(gru_hidden_size, num_classes)

#     def forward(self, x):
#         efficientnet_outputs = []
#         for image in x:
#             efficientnet_output = self.efficientnet(image)
#             efficientnet_output = efficientnet_output.view(efficientnet_output.size(0), -1, efficientnet_output.size(1))
#             efficientnet_outputs.append(efficientnet_output)
#         # Pasa cada imagen a través de EfficientNet y obtén los feature maps
#         efficientnet_output = self.efficientnet.extract_features(x)
#         efficientnet_output = efficientnet_output.view(efficientnet_output.size(0), -1, self.efficientnet_output_size)

#         # Pasa los feature maps de las imágenes a lo largo de la dimensión del tiempo (secuencia)
#         gru_input = efficientnet_output

#         # Pasa la secuencia de entrada a través de GRU
#         gru_output, _ = self.gru(gru_input)

#         # Pasa la última salida de la GRU a través de una capa completamente conectada
#         output = self.fc(gru_output[:, -1, :])
#         return output


In [9]:
if torch.cuda.is_available():
    print('GPU está disponible')
else:
    print('No se encontró GPU, usando CPU')

GPU está disponible


In [10]:
# Crea una instancia del modelo combinado
combined_model = CombinedModel(vgg_output_size, gru_hidden_size, gru_num_layers, num_classes)

c:\Users\Daniel\Main\UVG\Semestre VIII\Data science\Proyecto2\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Daniel\Main\UVG\Semestre VIII\Data science\Proyecto2\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# # Cargar el modelo EfficientNet preentrenado (por ejemplo, EfficientNet-B0)
# efficientnet_model = EfficientNet.from_pretrained('efficientnet-b0')

# # Cambiar el modelo combinado para usar EfficientNet
# combined_model = CombinedModel(efficientnet_model, gru_hidden_size,
#                                 gru_num_layers, num_classes)

In [12]:
combined_model.to('cuda')

CombinedModel(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      

In [13]:
# Define la función para preprocesar las imágenes para VGG16 y CombinedModel
def preprocess_image_for_combined_model(image):
    # Transformaciones para preprocesar las imágenes para VGG16
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Cambiar el tamaño a 224x224 (tamaño de entrada de la VGG16)
        transforms.ToTensor(),  # Convertir a tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización requerida por VGG16
    ])

    preprocessed_image = transform(image)  # Aplica las transformaciones
    return preprocessed_image

In [14]:
class ImageDataGenerator:
    def __init__(self, df, ct_folder, batch_size=32):
        self.df = df
        self.ct_folder = ct_folder
        self.batch_size = batch_size
        self.num_samples = len(df)
        self.current_idx = 0

    def __iter__(self):
        return self

    def __next__(self):
        batch_images = []
        batch_labels = []

        for _ in range(self.batch_size):
            if self.current_idx >= self.num_samples:
                self.current_idx = 0
                raise StopIteration

            ct_name = self.df.iloc[self.current_idx]['StudyInstanceUID']
            labels = self.df.iloc[self.current_idx][['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].tolist()

            with ZipFile(os.path.join(self.ct_folder, ct_name + '.zip'), 'r') as zip_ref:
                image_files = zip_ref.namelist()

                # Read and process each image
                ct_images = []
                for image_file in image_files:
                    with zip_ref.open(image_file) as img_file:
                        image = Image.open(img_file)  # Load the image
                        image = preprocess_image_for_combined_model(image)
                        ct_images.append(image)

                # Append images and labels to the batch
                batch_images.append(ct_images)
                batch_labels.append(labels)

            self.current_idx += 1

        
        return np.array(batch_images), np.array(batch_labels)

In [15]:
batch_size = 1

In [16]:
data_generator = ImageDataGenerator(data_VGG, 'imagenes_train', batch_size=batch_size)

In [17]:
# Especifica la carpeta donde se almacenarán los registros de TensorBoard
log_dir = "logs"

# Inicializa TensorBoard
writer = SummaryWriter(log_dir=log_dir)

In [18]:
# Hiperparámetros de entrenamiento
learning_rate = 0.001
num_epochs = 5  # Número de épocas (iteraciones completas sobre el conjunto de datos)
display_step = 10
# Suponiendo que `data_generator` es la instancia del generador de datos que creamos antes

# Definir la función de pérdida y el optimizador
criterion = WeightedMultiLabelLogLoss()  # Suponiendo una tarea de clasificación
optimizer = optim.Adam(combined_model.parameters(), lr=learning_rate)

# Entrenamiento del modelo
combined_model.train()  # Poner el modelo en modo de entrenamiento

for epoch in range(num_epochs):
    total_loss = 0.0
    running_loss = 0.0
    counter = 0
    # Iterar a través del generador en mini lotes
    for i, (images_batch, labels_batch) in enumerate(data_generator):
        # Convertir el lote de imágenes a tensores y pasarlos por el modelo
        images_tensor = torch.tensor(images_batch, dtype=torch.float32).to('cuda')
        labels_tensor = torch.tensor(labels_batch, dtype=torch.long).to('cuda')  # Usar torch.long para las etiquetas
        print("pase")
        # Realizar la propagación hacia adelante (forward pass)
        predictions = combined_model(images_tensor)

        loss = criterion(predictions, labels_tensor)

        # Realizar la retropropagación y la optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Acumular la pérdida total
        total_loss += loss.item()
        running_loss += loss.item()

        # Actualizar la pérdida en tiempo real en TensorBoard
        if i % display_step == display_step - 1:
            avg_loss = running_loss / display_step
            writer.add_scalar('Loss', avg_loss, epoch * len(data_generator) + i)

            running_loss = 0.0
        counter += 1
        if counter == 100:
            break

    # Calcular la pérdida promedio para la época
    average_loss = total_loss / len(data_generator)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}')

print('Entrenamiento completado.')

pase
pase


KeyboardInterrupt: 

In [ ]:
# Supongamos que ya has entrenado el modelo y deseas guardarlo
torch.save(combined_model.state_dict(), 'modelo_entrenado.pth')

In [ ]:
# Cargar el modelo entrenado
combined_model = CombinedModel(vgg_output_size, gru_hidden_size, gru_num_layers, num_classes)
combined_model.load_state_dict(torch.load('modelo_entrenado.pth'))